# Steam sale!

## Importera beatuifulsoup och urlrequest from urllib

In [863]:
import bs4 #beautifulSoup4
from urllib.request import urlopen as uReq #URL openmodule
from bs4 import BeautifulSoup as soup #BS module

## Här är mitt försök att söka upp 5 sidår

### 1. Bygg urlen och parse den

### 2.  Gör medsamma en filereader för att bygga upp en excell fil av all data

### 3. Gör containers variabeln som är hela källkoden inom den givna containern "div class"

### 4. Looppa igenom allting 5 gånger

In [867]:
#urllib hämtar websidan och BS parsar HTML

#Här använder jag en loop inne i en annan loop, min plan var att incrementera urlen med 1, men också
#att incrementera den först efter att den hämtat allt info från den första sidan. 

#Spara till csv och namnge filen
filename = "steam.csv"

#Öppna filen i "writemode"
f = open(filename, "w")

#Gör en variabel för att fylla på första raden av excell filen
headers = "Title|Rating|#Reviews|Discount%|Discountprice|Originalprice|Releasedate|Windows|Linux|Mac\n"
f.write(headers)

for page_number in range(1, 6):
    my_url = ("https://store.steampowered.com/search/?specials=1&page=%(d)d" % { 'd' : page_number}) 
    print(my_url)
    
    #ladda ner sidan som en klient
    uClient = uReq(my_url)
    page_html = uClient.read() #Dumpa rå HTML i en variabel
    uClient.close()
    
    #Parse HTML
    page_soup = soup(page_html, "html.parser")
    #print(my_url)
    
    containers = page_soup.findAll("div", {"class":"responsive_search_name_combined"})

    #Looppa igenom hela source-coden "containers" och looppa igenom all info
    for container in containers:
        #Title
        product_title = container.find("span", {"class", "title"}).text

        #Release date
        product_releasedate = container.find("div",  {"class", "col search_released responsive_secondrow"}).text    

        #Original price
        #Här hamnar jag ha en try & except därför att alla priser helt enkelt inte har rabatt (<strike> taggen)
        try:
            product_original_price = container.find("div", {"class", "col search_price discounted responsive_secondrow"}).find("strike").text
        #Discount % and price
            product_discount = container.find("div", {"class", "col search_discount responsive_secondrow"}).find("span").text
            product_price = container.find("div", {"class", "col search_price discounted responsive_secondrow"}).text
            product_discount_price = product_price.split("€", 1)[-1]
        except:
            print("no discount price!")

        #Platforms
        product_platform = container.find('div',{'class':'col search_name ellipsis'})
        platform_container = product_platform.find('p')
        if platform_container.find('span',{'class':'platform_img win'}):
            win = "1"
        else: win = "0"
        if platform_container.find('span',{'class':'platform_img mac'}):
            mac = "1"
        else: mac = "0"
        if platform_container.find('span',{'class':'platform_img linux'}):
            lin = "1"
        else: lin = "0"

        try:
            #här hamnar jag try & except därför att alla priser inte har reviews
            product_result_details = container.find("div", {"class", "col search_reviewscore responsive_secondrow"}).find("span")
            product_result = product_result_details['data-tooltip-html']

            #Rating
            product_rating_split = product_result.split("%")[0]
            product_rating = product_rating_split.split("<br>")[-1]

            #Amount of reviews
            product_review_split = product_result.split("the ")[-1]
            product_review = product_review_split.split("user")[0]
        except:
            print("None")

        f.write(product_title+"|"+product_rating+"% "+"|"+product_review+"|"+product_discount+"|"+product_discount_price+"|"+product_original_price+"|"+product_releasedate+"|"+win+"|"+lin+"|"+mac+"\n")
        #print(product_result_details)


https://store.steampowered.com/search/?specials=1&page=1
None
None
None
https://store.steampowered.com/search/?specials=1&page=2
no discount price!
None
https://store.steampowered.com/search/?specials=1&page=3
None
None
None
None
None
None
https://store.steampowered.com/search/?specials=1&page=4
None
None
None
None
None
no discount price!
None
None
None
https://store.steampowered.com/search/?specials=1&page=5
None
None
None
None
None
None
None
None
None
None


# Här är den mer simpla med en hårdkodad url

## Bygg urlen och parse den

In [852]:
my_url = ("https://store.steampowered.com/search/?specials=1&page=1") 
#print(my_url)
    
#ladda ner sidan som en klient
uClient = uReq(my_url)
page_html = uClient.read() #Dumpa rå HTML i en variabel
uClient.close()
    
#Parse HTML
page_soup = soup(page_html, "html.parser")
#print(my_url)

## Gör containers variabeln som är hela källkoden inom den givna containern "div class"
### Sök också alla containers med produktinfo (kålla längden)

In [853]:
containers = page_soup.findAll("div", {"class":"responsive_search_name_combined"})
len(containers) #längde containers

25

## Kålla vad som finns i först containern (hjälpmedel för att avläsa koden)

In [854]:
container = containers[3]
container

<div class="responsive_search_name_combined">
<div class="col search_name ellipsis">
<span class="title">Valley</span>
<p>
<span class="platform_img win"></span><span class="platform_img mac"></span><span class="platform_img linux"></span> </p>
</div>
<div class="col search_released responsive_secondrow">24 Aug, 2016</div>
<div class="col search_reviewscore responsive_secondrow">
<span class="search_review_summary positive" data-tooltip-html="Very Positive&lt;br&gt;90% of the 3,297 user reviews for this game are positive.">
</span>
</div>
<div class="col search_price_discount_combined responsive_secondrow">
<div class="col search_discount responsive_secondrow">
<span>-90%</span>
</div>
<div class="col search_price discounted responsive_secondrow">
<span style="color: #888888;"><strike>19,99€</strike></span><br/>1,99€							</div>
</div>
</div>

## Hitta variablar med att looppa igenom parsningen

### Gör medsamma en filereader för att bygga upp en excell fil av all data

In [856]:
#Spara till csv och namnge filen
filename = "steam.csv"

#Öppna filen i "writemode"
f = open(filename, "w")

#Gör en variabel för att fylla på första raden av excell filen
headers = "Title|Rating|#Reviews|Discount%|Discountprice|Originalprice|Releasedate|Windows|Linux|Mac\n"
f.write(headers)

#Looppa igenom hela source-coden "containers" och looppa igenom all info
for container in containers:
    #Title
    product_title = container.find("span", {"class", "title"}).text
    
    #Release date
    product_releasedate = container.find("div",  {"class", "col search_released responsive_secondrow"}).text    
    
    try:
        #Original price
        product_original_price = container.find("div", {"class", "col search_price discounted responsive_secondrow"}).find("strike").text
        
        #Discount % and price
        product_discount = container.find("div", {"class", "col search_discount responsive_secondrow"}).find("span").text
        product_price = container.find("div", {"class", "col search_price discounted responsive_secondrow"}).text
        product_discount_price = product_price.split("€", 1)[-1]
    except:
        print("No discount!")
    
    #Platforms
    product_platform = container.find('div',{'class':'col search_name ellipsis'})
    platform_container = product_platform.find('p')
    if platform_container.find('span',{'class':'platform_img win'}):
        win = "1"
    else: win = "0"
    if platform_container.find('span',{'class':'platform_img mac'}):
        mac = "1"
    else: mac = "0"
    if platform_container.find('span',{'class':'platform_img linux'}):
        lin = "1"
    else: lin = "0"
    
    try:
        product_result_details = container.find("div", {"class", "col search_reviewscore responsive_secondrow"}).find("span")
        product_result = product_result_details['data-tooltip-html']
        
        #Rating
        product_rating_split = product_result.split("%")[0]
        product_rating = product_rating_split.split("<br>")[-1]
        
        #Amount of reviews
        product_review_split = product_result.split("the ")[-1]
        product_review = product_review_split.split("user")[0]
    except:
        print("None")
     
    f.write(product_title+"|"+product_rating+"% "+"|"+product_review+"|"+product_discount+"|"+product_discount_price+"|"+product_original_price+"|"+product_releasedate+"|"+win+"|"+lin+"|"+mac+"\n")
    #print(product_result_details)
    

None
None
None


# Slutsatser

Uppgifter var mycket lärorik och rolig att göra. Mest hade jag problem med att få u reviews att fungera. Då jag skulle fånga ['data-tooltip-html'], som inte fanns i alla containers. Det gjorde mitt program att crasha och fast jag satte in dem i en try & except, så gav den länge error. 

Efter jag satt in try & except i min loop så visade den "None" vid varje container som saknade reviews, fasten jag sökte efter title eller price. 